In [1]:
import numpy as np
import pandas as pd
from numba import jit

/tmp/ipykernel_18832/2995603844.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Exploring Chain 

In [44]:
from abc import ABC, abstractmethod
from functools import lru_cache


@lru_cache
@jit(nopython=True)
def get_geometric_probs(n: int, p: int) -> np.ndarray:
    vector = np.array([p]) ** np.arange(1, n+1)
    return vector / np.sum(vector)


class MarkovChain(ABC):
    def __init__(self, n_cities: int) -> None:
        self.n_cities = n_cities
        assert self.n_cities > 0
        self.rng  = np.random.default_rng()

    @abstractmethod
    def next_state(self, state: np.ndarray) -> np.ndarray:
        ...


class RandomWalk(MarkovChain):
    def __init__(self, n_cities: int) -> None:
        super().__init__(n_cities)

    def next_state(self, state: np.ndarray) -> np.ndarray:
        u = self.rng.random()
        if u <= (1 / (self.n_cities + 1)):
            return state
        
        idx = self.rng.integers(0, self.n_cities, endpoint=False)
        new_state = state.copy()
        new_state[idx] = ~new_state[idx]
        return new_state


class WeightedRandomJumps(MarkovChain):
    def __init__(self, n_cities: int) -> None:
        super().__init__(n_cities)

    def next_state(self, state: np.ndarray) -> np.ndarray:
        n = self.n_cities
        probs = get_geometric_probs(n, 1/2)
        m = self.rng.choice(n, p=probs) + 1
        idxs = self.rng.choice(n, m, replace=False)
        
        new_state = state.copy()
        new_state[idxs] = ~new_state[idxs]
        return new_state

# Metropolis-Hastings

In [45]:
def initalize_random_state(n) :
    return np.random.random(n) < 0.5

In [159]:
from typing import Optional

class MetropolisHastings:
    states: list[np.ndarray]
    scores: np.ndarray

    def __init__(self, n_cities: int, n_chains: int, exploring_chain: MarkovChain, objective_function, states: Optional[list[np.ndarray]] = None):
        self.n_cities = n_cities
        self.n_chains = n_chains
        self.rng = np.random.default_rng()
        self.exploring_chain = exploring_chain
        self.objective_function = objective_function
        
        if states is not None:
            self.states = states
        else:
            self.states = [
                initalize_random_state(n_cities)
                for _ in range(n_chains)
            ]

        self.states = np.array(self.states, dtype=np.bool_)

        self.scores = [
            self.objective_function(state)
            for state in self.states
        ]
        self.scores = np.array(self.scores)

    def acceptance(self, prev_score, new_score):
        return min(np.exp(new_score - prev_score), 1)

    def forward(self):
        for i, state in enumerate(self.states):
            new_state = self.exploring_chain.next_state(state)
            new_score = self.objective_function(new_state)
            u = self.rng.random()
            if u <= self.acceptance(self.scores[i], new_score):
                self.states[i] = new_state
                self.scores[i] = new_score


# Solver

In [160]:
@jit(nopython=True)
def calculate_radius(distance_matrix: np.ndarray, state: np.ndarray) -> float:
    return distance_matrix[state].T[state].max() / 2

In [161]:
from typing import Callable

class Solver:
    def __init__(self, populations: np.ndarray, distance_matrix: np.ndarray, exploring_chain_type: type[MarkovChain]):
        self.populations = populations
        self.distance_matrix = distance_matrix

        self.n_cities = len(populations)
        self.exploring_chain = exploring_chain_type(n_cities=self.n_cities)
        assert self.distance_matrix.shape == (self.n_cities, self.n_cities)

    def get_objective_funtion(self, lambda_: float) -> Callable[[np.ndarray], float]:

        def objective_function(state: np.ndarray) -> float:
            r = calculate_radius(self.distance_matrix, state)
            value = np.sum(self.populations, where=state) - lambda_ * self.n_cities * np.pi * r ** 2
            return value

        return objective_function

    def simulate_chains(self, lambda_: float = 0.1, n_chains: int = 10, steps: int = 1000):

        mh = MetropolisHastings(
            n_cities=self.n_cities,
            n_chains=n_chains,
            exploring_chain=self.exploring_chain,
            objective_function=self.get_objective_funtion(lambda_)
        )

        best_state = None
        best_score = -np.inf

        step_best_scores = []
        step_best_state_size = []
        for _ in range(steps):
            mh.forward()
            idx = np.argmax(mh.scores)
            step_best_scores.append(mh.scores[idx])
            step_best_state_size.append(mh.states[idx].sum())
            if mh.scores[idx] > best_score:
                best_state = mh.states[idx]
                best_score = mh.scores[idx]

        return {
            "final_states": mh.states,
            "best_score": best_score,
            "best_state": best_state,
            "step_best_scores": step_best_scores,
            "step_best_state_size": step_best_state_size
        }


# Example

In [162]:
populations = np.random.random(100)
positions = np.random.random((100, 2))

distance_matrix = np.zeros((100, 100), dtype=np.float32)
for i in range(100):
    for j in range(i, 100):
        distance_matrix[i, j] = distance_matrix[j, i] = np.sum((positions[i] - positions[j]) ** 2) ** 0.5

In [163]:
solver = Solver(populations, distance_matrix, RandomWalk)
solver.simulate_chains(1, 100, 10000)["best_score"]

0.05597096309830585

In [164]:
import cProfile

In [167]:
solver = Solver(populations, distance_matrix, RandomWalk)
with cProfile.Profile() as pf:
    pf.run('solver.simulate_chains(1, 1000, 1000)["best_score"]')

In [168]:
pf.print_stats(sort="cumtime")

         16002359 function calls in 22.585 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   22.585   22.585 cProfile.py:93(run)
        1    0.000    0.000   22.585   22.585 cProfile.py:98(runctx)
        1    0.000    0.000   22.585   22.585 {built-in method builtins.exec}
        1    0.000    0.000   22.585   22.585 <string>:1(<module>)
        1    0.008    0.008   22.585   22.585 3330448856.py:21(simulate_chains)
     1000    1.788    0.002   22.544    0.023 3436739998.py:33(forward)
  1001000    2.459    0.000   13.341    0.000 3330448856.py:14(objective_function)
  1001000    5.914    0.000    5.914    0.000 3160897994.py:1(calculate_radius)
  1000000    1.797    0.000    5.210    0.000 578288408.py:27(next_state)
  1001000    0.840    0.000    4.841    0.000 fromnumeric.py:2177(sum)
  1001000    1.084    0.000    3.841    0.000 fromnumeric.py:71(_wrapreduction)
   990162    2.430    0.

In [ ]:
%%timeit
np.random.random(100)

1.44 µs ± 96.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
%%timeit
rgn.random(100)

1.07 µs ± 34.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
# Generating 1 random int in a range takes 6.61 mirco sec
# Generating 20_000 random int in a range takes 20.1 micro sec